In [0]:
!pip3 install pytorch-nlp
from torchnlp.word_to_vector import GloVe
g_vec = GloVe(name='6B', dim='50')

In [0]:
import numpy as np

g_vocab = g_vec.token_to_index.keys()
g_vocab = np.array(list(g_vocab))

import nltk

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()

import re
from tqdm.notebook import tqdm

def retokenize(t) :
  if type(t) == type("str") :
    tt = t
  elif type(t) == type(list()) :
    tt = ' '.join(t)
  elif type(t) == type(np.array(list())) :
    tt = ' '.join(t.tolist())
  else :
    return False
  return np.array(nltk.tokenize.word_tokenize(tt))

def researcher(word) :
  i = 0
  right = True
  for i in range(len(word)) :
    if word[i:] in g_vocab :
      break
    if word[:-i] in g_vocab :
      right = False
      break
  new_word = word[i:] if right else word[:-i]
  if len(new_word) < 3 :
    new_word = '<UNK>'
  return new_word
  

def clean_text(name) :
  t = open('./'+name+'.txt').read()
  t = t.lower()
  t = retokenize(t)
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(lambda w: re.sub('//\S*', 'url', w), tqdm(t[not_found])))
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(lambda w: re.sub("(\d+)", ' \1 ', w), tqdm(t[not_found])))
  t = retokenize(t)
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(lambda w: re.sub("[^a-zA-Z0-9|\-|']", ' ', w), tqdm(t[not_found])))
  t = retokenize(t)
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(lambda w: re.sub("[\-|']", ' ', w), tqdm(t[not_found])))
  t = retokenize(t)
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(stemmer.stem, tqdm(t[not_found])))
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(lemmatizer.lemmatize, tqdm(t[not_found])))
  t = retokenize(t)
  not_found = ~np.isin(t, g_vocab)
  t[not_found] = list(map(researcher, tqdm(t[not_found])))
  t = t[t != '']
  return t.tolist()

train_text = clean_text('train')
test_text = clean_text('test')

import json

json.dump(train_text, open('clean_train.json','w'))
json.dump(test_text, open('clean_test.json','w'))

vocab = np.unique(train_text+test_text)

In [0]:
from tqdm.notebook import tqdm
import json

vocabl = vocab.tolist()
i_train_text = list(map(vocabl.index, tqdm(train_text)))
i_test_text = list(map(vocabl.index, tqdm(test_text)))

json.dump(i_train_text, open('i_train.json','w'))
json.dump(i_test_text, open('i_test.json','w'))
del vocabl